In [18]:
from lxml import html as lhtml                         # Непосредственно для парсинга.
import requests                                        # Для получения html кода сайта.
from multiprocessing.dummy import Pool as ThreadPool   # Для реализации параллельной обкачки.
from math import ceil                                  # Для округления вверх.
from tqdm.notebook import tqdm                         # Для визуализации прогресса итерации по генератору.
import time                                            # Для ожидания окончания работы в  get_page()

import numpy as np  # Для дебага, чтобы получать произвольные url'ы.
import random       # Для дебага, чтобы получать произвольные url'ы.

import pandas as pd # Непосредственно для формирования *.csv файла.

In [2]:
with open('authors.txt', 'r') as f:
    authors_id = list(map(int, f.read().split('\n')[:-1]))

In [4]:
way = '//div[@class="rd-listing-product-item rd-listing-product-item_size- rd-listing-product-item_type-"]/a'

In [29]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    r_get = requests.get(url, **kwargs)
    if not r_get.ok:
        time.sleep(t_sleep)
        if n_attempts > 0:
            return get_page(url, n_attempts-1, t_sleep, kwargs)
    return r_get   

In [30]:
def get_url(author):
    
    url_for_download = []

    page_url = "https://www.respublica.ru/authors/" + str(author)
    html = get_page(page_url)
    tree = lhtml.fromstring(html.text)
    all_page_nums = ceil(int(tree.xpath('//span[@class="rd-listing-count__total"]')[0].text) / 22)

    for page_num in range(all_page_nums):
        
        page_url = "https://www.respublica.ru/authors/" + str(author)
        html = get_page(page_url, params={'page': page_num, })
        tree = lhtml.fromstring(html.text)
        all_book_on_page = tree.xpath(way)

        for book in all_book_on_page:
            url_for_download.append("https://www.respublica.ru"+book.attrib['href'])
    return url_for_download

In [31]:
%%time

with ThreadPool(10) as pool:
    authors_book_urls = pool.map(get_url, authors_id)
pool.join()

url_history_for_download = [url for sub_list in authors_book_urls for url in sub_list]

CPU times: user 5.42 s, sys: 161 ms, total: 5.58 s
Wall time: 10.6 s


In [32]:
len(url_history_for_download)

2860

In [33]:
charact = ["ISBN", "Издательство", "Серия", "Обложка", "Формат", "Количество страниц",\
           "Год издания", "Язык", "Вес, г", "Направление", "Возрастные ограничения", "Страна-производитель"]

template = {key: None for key in charact}

In [34]:
# indexes = np.random.randint(0,len(url_history_for_download), 100)

In [35]:
charact = ["ISBN", "Издательство", "Серия", "Обложка", "Формат", "Количество страниц",\
           "Год издания", "Язык", "Вес, г", "Направление", "Возрастные ограничения", "Страна-производитель"]
template = {key: None for key in charact}

def get_site(url):
    
    field = []
    html = get_page(url)
    tree = lhtml.fromstring(html.text)
    tmp = tree.xpath('//div[@class="rd-page-breadcrumbs rd-page-product__breadcrumbs"]//span[@itemprop="name"]//text()')
    
    field.append(url) # 0 - URL.
    field.append([item[1:] for item in tmp]) #1 - Категория.
    field.append(tree.xpath('//h1[@class="rd-page-product__title"]//text()')) #2 - Название.
    field.append(tree.xpath('//a[@itemprop="brand"]//text()')[0]) # 3 - Автор.
    field.append(tree.xpath('//span[@itemprop="sku"]')[0].text) # 4 - Артикул.
    
    tmp = tree.xpath('//a[@class="download-pdf"]')
    if bool(tmp):
        tmp = tmp[0].attrib['href']
        field.append("https://www.respublica.ru" + tmp) # 5 - Превью.
    else:
        field.append(None)
        
    field.append(tree.xpath('//meta[@property="og:image"]')[0].attrib['content']) # 6 - Изображение.

    total_mark     = len(tree.xpath('//div[@class="rd-page-product__rating-stars"]//div[@class="rd-rating-stars__star rd-rating-stars__star_fill"]'))
    comment_number = len(tree.xpath('//div[@class="rd-product-reviews__item"]//div[@class="rd-product-reviews__item-body"]'))
    mark_number    = len(tree.xpath('//div[@class="rd-product-reviews__item"]//div[@class="rd-rating-stars"]'))
    field.append([total_mark, comment_number, mark_number]) # 7 - Оценка, отзывы, оценки.
    
    field.append(int(tree.xpath('//span[@class="num"]//text()')[0].replace(' ',''))) # 8 - Цена.    
    
    tmp = tree.xpath('//div[@class="rd-page-product__price-old"]//span//text()')
    if bool(tmp):
        field.append(tmp[0]) # 9 - старая Цена.
    else:
        field.append(None)
        
    field.append(bool(tree.xpath('//a[@class="rd-page-product__buy rd-page-product__buy_status_available"]'))) # 10 - Наличие.
    field.append(' '.join(tree.xpath('//div[@class="rd-page-product__desc-body"]//text()'))) # 11 - Описание.

    tmp = tree.xpath('//p[@class="rd-page-product__desc-param"]//text()')
    tmp = {tmp[i]: tmp[i+2] for i in range(0,len(tmp)-1,3)}
    temp = template.copy()
    for key in tmp.keys():
        temp[key] = tmp[key]
    field.append(temp) # 12 - Характеристики.    

    return field

In [36]:
%%time

with ThreadPool(100) as pool:
    all_sites = pool.map(get_site, url_history_for_download)
pool.join()

CPU times: user 1min 14s, sys: 3.44 s, total: 1min 18s
Wall time: 2min 55s


In [37]:
len(all_sites)

2860

In [39]:
import pandas as pd
sites = pd.DataFrame()

for site in tqdm(all_sites):
    
    tmp_frame = pd.DataFrame({'url':           site[0],
                              'Название':      site[2],
                              'Автор':         site[3],
                              'Превью':        site[5],
                              'Изображение':   site[6],
                              'Описание':      site[11],
                              'Цена':          site[8],
                              'Цена (старая)': site[9],
                              'В наличии':     site[10],
                              'Категория':     '; '.join(site[1]),
                              'Число отзывов': site[7][1],
                              'Число оценок':  site[7][2],
                              'Оценка':        site[7][0],
                              'ISBN':          site[12]['ISBN'],
                              'Издательство':  site[12]['Издательство'],
                              'Серия':         site[12]['Серия'],
                              'Обложка':       site[12]['Обложка'],
                              'Формат':        site[12]['Формат'],
                              'Количество страниц':     site[12]['Количество страниц'],
                              'Год издания':            site[12]['Год издания'],
                              'Язык':                   site[12]['Язык'],
                              'Возрастные ограничения': site[12]['Возрастные ограничения'],
                              'Направление':            site[12]['Направление'],
                              'Страна-производитель':   site[12]['Страна-производитель']}, index=[site[4]])
    
    sites = sites.append(tmp_frame)
    
sites.head(10)

,url,Название,Автор,Превью,Изображение,Описание,Цена,Цена (старая),В наличии,Категория,...,Издательство,Серия,Обложка,Формат,Количество страниц,Год издания,Язык,Возрастные ограничения,Направление,Страна-производитель
513925,https://www.respublica.ru/knigi/hudozhestvenna...,Чужак,Стивен Кинг,https://www.respublica.ru/items/373815/downloa...,https://www.respublica.ru/uploads/00/00/00/bz/...,В парке маленького городка Флинт-Сити найден т...,370,460,False,Книги; Художественная литература; Фантастика и...,...,АСТ,Темная башня,Твердая,"20 x 12,5",576,2019,Русский,12+,Мистика,Россия
510392,https://www.respublica.ru/knigi/hudozhestvenna...,На подъеме,Стивен Кинг,https://www.respublica.ru/items/370007/downloa...,https://www.respublica.ru/uploads/01/00/00/bf/...,\r\n\tКасл-Рок — маленький городок в штате Мэн...,290,None,True,Книги; Художественная литература; Фантастика и...,...,АСТ,Вселенная Стивена Кинга,Твердая,"16,5 x 12,5",192,2018,Русский,None,Мистика,Россия
486522,https://www.respublica.ru/knigi/hudozhestvenna...,Противостояние. 5 июля 1990 - 10 января 1991,Стивен Кинг,https://www.respublica.ru/items/361968/downloa...,https://www.respublica.ru/uploads/00/00/00/6x/...,Америка превратилась в ад. Из секретной лабора...,260,None,True,Книги; Художественная литература; Фантастика и...,...,АСТ,Король на все времена(м),Мягкая,"11,5 x 18",992,2018,Русский,16+,Мистика,Россия
484943,https://www.respublica.ru/knigi/hudozhestvenna...,"Девочка, которая любила Тома Гордона",Стивен Кинг,https://www.respublica.ru/items/360505/downloa...,https://www.respublica.ru/uploads/01/00/00/6o/...,"Девятилетняя Триша заблудилась в лесу, и чем д...",170,None,True,Книги; Художественная литература; Фантастика и...,...,АСТ,Современная зарубежная проза,Мягкая,12 х 18,288,2018,Русский,0+,Фэнтези,Россия
483456,https://www.respublica.ru/knigi/hudozhestvenna...,Мешок с костями,Стивен Кинг,https://www.respublica.ru/items/359228/downloa...,https://www.respublica.ru/uploads/01/00/00/6m/...,"Добро пожаловать в маленький тихий городок, ку...",340,None,True,Книги; Художественная литература; Фантастика и...,...,АСТ,Король на все времена,Мягкая,12 х 18,704,2018,Русский,16+,Мистика,Россия
482933,https://www.respublica.ru/knigi/hudozhestvenna...,После заката,Стивен Кинг,https://www.respublica.ru/items/358652/downloa...,https://www.respublica.ru/uploads/00/00/00/6l/...,"""Команда скелетов"", ""Ночная смена"", ""Все преде...",540,None,True,Книги; Художественная литература; Классическая...,...,АСТ,Темная башня,Твердая,"20,5 х 13",416,2018,Русский,None,None,Россия
480929,https://www.respublica.ru/knigi/hudozhestvenna...,Секретные окна,Стивен Кинг,https://www.respublica.ru/items/356645/downloa...,https://www.respublica.ru/uploads/01/00/00/6i/...,Вы хотите познакомиться с двумя самыми первыми...,360,None,True,Книги; Художественная литература; Фантастика и...,...,АСТ,Вселенная Стивена Кинга,Твердая,13 х 17,544,2018,Русский,16+,Фэнтези,Россия
471244,https://www.respublica.ru/knigi/hudozhestvenna...,Мешок с костями,Стивен Кинг,https://www.respublica.ru/items/346936/downloa...,https://www.respublica.ru/uploads/01/00/00/61/...,"Добро пожаловать в маленький тихий городок, ку...",340,None,True,Книги; Художественная литература; Фантастика и...,...,АСТ,Король на все времена,Твердая,13 x 20,640,2018,Русский,6+,Мистика,Россия
470118,https://www.respublica.ru/knigi/hudozhestvenna...,Кэрри,Стивен Кинг,https://www.respublica.ru/items/345815/downloa...,https://www.respublica.ru/uploads/00/00/00/62/...,Маленький провинциальный городок в Новой Англи...,170,260,False,Книги; Художественная литература; Фантастика и...,...,АСТ,Король на все времена,Мягкая,11 х 18,256,2018,Русский,0+,Мистика,Россия
470123,https://www.respublica.ru/knigi/hudozhestvenna...,Пляска смерти,Стивен Кинг,https://www.respublica.ru/items/345810/downloa...,https://www.respublica.ru/uploads/01/00/00/62/...,Одна из наиболее известных нехудожественных ра...,440,None,True,Книги; Художественная литература; Фантастика и...,...,АСТ,Темная 

In [40]:
sites.shape

(2860, 24)

In [41]:
sites.to_csv('d.harazyan_hw_3.csv')